In [1]:
import os
import numpy as np
import glob
import pandas as pd  
import random
import math as mp
import pickle
#import matplotlib.pyplot as plt
import tensorflow as tf
import time
from pickle import load

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

%matplotlib inline
#import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow.keras.backend as K

from pickle import dump

from numpy import array
from numpy import hstack
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM, CuDNNGRU
from tensorflow.keras.layers import Dense,Flatten,Dropout, MaxPooling1D

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping


from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from pyquaternion import Quaternion

import toml

from math import floor



In [5]:
######################################################################
# Init Configuration
######################################################################

config_path = 'C:/Users/weikert1/Documents/Thesis/Project_V06_26072021/Code/V06_config.toml'
cfg = toml.load(config_path)

# General 
n_in_seq = cfg['n_in_seq_euler']
n_out_seq = cfg['n_out_seq_euler']
n_steps = cfg['n_steps'] 
LAT_in_rows = cfg['LAT_in_rows'] #6*16,66ms= 64ms

# model ID --> specify this for every new model
model_id = cfg['4_2_model_ID']
model_type = cfg['4_model_type']

# Directories 
Path_dataset01_test = cfg['Path_dataset01']
Path_results_dataset01_GRU_euler_model_id = cfg['Path_results_dataset01'] + model_type + "/" + model_id  + "/" 

# Tensorflow configuration
#config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.compat.v1.Session(config=config)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))


Num GPUs Available:  1


In [8]:
def load_df(path): 
    df = pd.read_csv(path, index_col=None)

    return df

def get_csv_files(dataset_path, keyword):
    """
    Generator function to recursively output the CSV files in a directory and its sub-directories.
    Arguments:
        dataset_path: Path to the directory containing the CSV files.
    Outputs:
        Paths of the found CSV files.
    """
    files = []
 
    for f in glob.glob(os.path.join(dataset_path,"*{}*".format(keyword))):
        #print(f)
        if not os.path.isdir(f):
            file_name, extension = f.split('.')            
            if extension == "csv":
                files.append(f)
            else:
                logging.warn("Invalid file: {}. Ignoring...".format(f))

    return files

def split_sequences(sequences, n_steps, LAT, n_in_seq, n_out_seq):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        # find the end of this pattern
        stop_ix = i + n_steps
        shift_ix = i + n_steps + LAT
        seq_length = len(sequences) - LAT
        if stop_ix > seq_length:
          break
        seq_x = sequences[i:stop_ix,:-n_in_seq]
        seq_y = sequences[shift_ix-1, -n_out_seq:]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)
    #return X, y


def get_sequence_quat(dataframe, dataset_path_train):
    # multivariate data preparation

    # define sequences   
    in_seq4 = dataframe[['qw']].to_numpy()
    in_seq5 = dataframe[['qx']].to_numpy()
    in_seq6 = dataframe[['qy']].to_numpy()
    in_seq7 = dataframe[['qz']].to_numpy()
    
    out_seq4 = dataframe[['qw']].to_numpy()
    out_seq5 = dataframe[['qx']].to_numpy()
    out_seq6 = dataframe[['qy']].to_numpy()
    out_seq7 = dataframe[['qz']].to_numpy()
    
    # convert to [rows, columns] structure   
    in_seq4 = in_seq4.reshape((len(in_seq4), 1))
    in_seq5 = in_seq5.reshape((len(in_seq5), 1))
    in_seq6 = in_seq6.reshape((len(in_seq6), 1))
    in_seq7 = in_seq7.reshape((len(in_seq7), 1))
    
    out_seq4 = out_seq4.reshape((len(out_seq4), 1))
    out_seq5 = out_seq5.reshape((len(out_seq5), 1))
    out_seq6 = out_seq6.reshape((len(out_seq6), 1))
    out_seq7 = out_seq7.reshape((len(out_seq7), 1))
        

    # horizontally stack columns
    #dataset = hstack((in_seq1, in_seq2, in_seq3, in_seq4, in_seq5, in_seq6, in_seq7, out_seq1, out_seq2, out_seq3, out_seq4, out_seq5, out_seq6, out_seq7))
    dataset = hstack((in_seq4, in_seq5, in_seq6, in_seq7,out_seq4, out_seq5, out_seq6, out_seq7))

    #scaler_obj = scaler(dataset)
    #normalized_dataset = scaler_obj.transform(dataset)
    #print(normalized_dataset)
    
    # define number of input and output sequences
    n_in_seq = 4
    n_out_seq = 4
        
    # choose a number of time steps
    n_steps = 15 
    
    dt = 5 # sampling rate in ms
    LAT_in_ms = 100
    LAT_in_rows = 5 #6*16,66ms= 64ms

    # convert into input/output
    X, y = split_sequences(dataset, n_steps, LAT_in_rows, n_in_seq, n_out_seq)

    print("X shape is: " + str(X.shape))
    print("y shape is: " + str(y.shape))
    print("------")
    
    return X,y

In [9]:
def prepare_training_data_quats(dataset_path_train):
 
    X_train_result = [] 
    y_train_result = [] 
    
    X_val_result = [] 
    y_val_result = [] 

    
    for csv in get_csv_files(dataset_path_train, "*train"):
        csv_df = load_df(csv)
        X_single, y_single = get_sequence_quat(csv_df, dataset_path_train)
        
        # split into X_single_train, X_single_test and y_single_train, y_single_test with 80/20 split
        train_size = int(len(X_single) * 0.80)
        test_size = len(X_single) - train_size
        X_train, X_val = X_single[0:train_size,:], X_single[train_size:len(X_single),:] 
        y_train, y_val =  y_single[0:train_size,:], y_single[train_size:len(y_single),:]     
        
        X_train_result.append(X_train)
        y_train_result.append(y_train)
        
        X_val_result.append(X_val)
        y_val_result.append(y_val)
        
    X_train_result = np.vstack((X_train_result))
    y_train_result = np.vstack((y_train_result))

    X_val_result = np.vstack((X_val_result))
    y_val_result = np.vstack((y_val_result))
        
    return X_train_result, X_val_result, y_train_result, y_val_result

In [10]:
######################################################################
# Load Training Data
######################################################################

#pd.set_option("display.precision", 2)
X_train, X_val, y_train, y_val = prepare_training_data_quats(Path_dataset01_test)

X shape is: (71405, 15, 4)
y shape is: (71405, 4)
------
